In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
indep=dataset[["Gender_Male","Age",'EstimatedSalary']]
dep=dataset[["Purchased"]]

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import CategoricalNB
param_grid={'alpha':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0], 'force_alpha':[True,False], 'fit_prior':[True,False], 'class_prior':[None]}
grid= GridSearchCV(CategoricalNB(), param_grid, refit= True, verbose=3, n_jobs=-1, scoring= 'f1_weighted' )
grid.fit(X_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Anaconda3run\Lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan]
  warnings.warn(
C:\Anaconda3run\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=CategoricalNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,
                                   1.0],
                         'class_prior': [None], 'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             scoring='f1_weighted', verbose=3)

In [10]:
y_pred=grid.predict(X_test)

In [11]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[77,  8],
       [15, 34]], dtype=int64)

In [12]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)

In [13]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.84      0.91      0.87        85
           1       0.81      0.69      0.75        49

    accuracy                           0.83       134
   macro avg       0.82      0.80      0.81       134
weighted avg       0.83      0.83      0.83       134



In [14]:
from sklearn.metrics import roc_auc_score
roc_score= roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])
roc_score

0.870828331332533

In [15]:
print ( " the best parameter is ", grid.best_params_)

 the best parameter is  {'alpha': 0.1, 'class_prior': None, 'fit_prior': True, 'force_alpha': True}


In [16]:
grid.cv_results_

{'mean_fit_time': array([0.02613163, 0.02971768, 0.02453284, 0.0227385 , 0.02154307,
        0.02592936, 0.02333903, 0.02254019, 0.02273817, 0.02333765,
        0.02193933, 0.02234154, 0.02434072, 0.02134199, 0.02154264,
        0.02433486, 0.02253494, 0.02513199, 0.02174401, 0.02573023,
        0.02473426, 0.02194328, 0.02512836, 0.0215436 , 0.02253809,
        0.02353706, 0.02373719, 0.02254057, 0.02473245, 0.02433467,
        0.0239306 , 0.02254004, 0.02353992, 0.02393274, 0.02333641,
        0.02353725, 0.02174067, 0.02353663, 0.02313795, 0.02553082]),
 'std_fit_time': array([0.00193713, 0.00899739, 0.00391827, 0.00263197, 0.00162013,
        0.00302675, 0.00224028, 0.00381624, 0.00116441, 0.00195509,
        0.00109232, 0.00101832, 0.00279827, 0.00079748, 0.00101692,
        0.00331452, 0.0019583 , 0.00486233, 0.001468  , 0.00353504,
        0.00353634, 0.00063446, 0.00353027, 0.0011961 , 0.00232596,
        0.0023258 , 0.00230717, 0.00264689, 0.00330315, 0.00331421,
        0.002

In [17]:
re=grid.cv_results_

In [18]:
table= pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_class_prior,param_fit_prior,param_force_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.026132,0.001937,0.020144,0.011006,0.1,None,True,True,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.735616,0.743112,0.710831,0.805463,NaN,NaN,NaN,1
1,0.029718,0.008997,0.012966,0.003337,0.1,None,True,False,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.735616,0.743112,0.710831,0.805463,NaN,NaN,NaN,1
2,0.024533,0.003918,0.011769,0.001717,0.1,None,False,True,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.719468,0.748608,0.732285,0.849057,NaN,NaN,NaN,1
3,0.022739,0.002632,0.014162,0.002919,0.1,None,False,False,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.719468,0.748608,0.732285,0.849057,NaN,NaN,NaN,1
4,0.021543,0.001620,0.012566,0.004214,0.2,None,True,True,"{'alpha': 0.2, 'class_prior': None, 'fit_prior...",0.735616,0.743112,0.710831,0.783837,NaN,NaN,NaN,1
5,0.025929,0.003027,0.012367,0.001353,0.2,None,True,False,"{'alpha': 0.2, 'class_prior': None, 'fit_prior...",0.735616,0.743112,0.710831,0.783837,NaN,NaN,NaN,1
6,0.023339,0.002240,0.010969,0.003888,0.2,None,False,True,"{'alpha': 0.2, 'class_prior': None, 'fit_prior...",0.719468,0.770070,0.732285,0.849057,NaN,NaN,NaN,1
7,0.022540,0.003816,0.011971,0.003570,0.2,None,False,False,"{'alpha': 0.2, 'class_prior': None, 'fit_prior...",0.719468,0.770070,0.732285,0.849057,NaN,NaN,NaN,1
8,0.022738,0.001164,0.011770,0.004008,0.3,None,True,True,"{'alpha': 0.3, 'class_prior': None, 'fit_prior...",0.751984,0.743112,0.688741,0.783837,NaN,NaN,NaN,1
9,0.023338,0.001955,0.013566,0.005487,0.3,None,True,False,"{'alpha': 0.3, 'class_prior': None, 'fit_prior...",0.751984,0.743112,0.688741,0.783837,NaN,NaN,NaN,1


In [19]:
grid.best_score_

nan

In [29]:
result= grid.predict([[0,35,60000]])
result

C:\Anaconda3run\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(


array([0], dtype=int64)